# Mushroom Classification with Deep Learning

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/FractalN/blob/main/Mushroom_Classifier.ipynb)

This notebook trains a mushroom classifier using transfer learning and data augmentation.

In [ ]:
# Check if we're running in Colab
IN_COLAB = 'google.colab' in str(get_ipython())

if IN_COLAB:
    # Clone repository (includes dataset in data/ folder)
    !git clone https://github.com/YOUR_USERNAME/FractalN.git
    %cd FractalN
    
    # Install additional requirements
    !pip install -r requirements.txt
    
    # Verify dataset is present
    !ls -R data/
    
    # Mount Google Drive for saving results
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Create results directory in Drive
    !mkdir -p "/content/drive/MyDrive/FractalN_Results"
else:
    # Verify dataset is present in local environment
    if not os.path.exists('data'):
        raise FileNotFoundError("Dataset not found! Please ensure 'data' directory exists.")
    print("Dataset found in local environment.")

In [ ]:
# Setup environment
import os
import sys
from src.gpu_config import setup_gpu
import tensorflow as tf

# Setup GPU
setup_gpu()

print("TensorFlow version:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices('GPU'))

# Set results directory based on environment
RESULTS_DIR = '/content/drive/MyDrive/FractalN_Results' if IN_COLAB else 'results'

## 1. Data Organization
First, we'll organize raw images into proper categories

In [ ]:
from src.utils.reorganize_data import reorganize_mushroom_data

# Organize raw data
reorganize_mushroom_data()

# Verify directory structure
!tree data/mushroom_data -L 2

## 2. Data Augmentation
Augment the dataset to improve model generalization

In [ ]:
from src.utils.augment_mushroom_data import augment_mushroom_data

# Augment data to reach target count per class
augment_mushroom_data(target_count=20000)

# Show class distribution after augmentation
!find data/mushroom_data/poisonous -type f | wc -l
!find data/mushroom_data/edible -type f | wc -l

## 3. Data Preprocessing
Preprocess and split the dataset into train/test sets

In [ ]:
from src.utils.preprocess_data import preprocess_dataset

# Preprocess and split dataset
preprocess_dataset(
    data_dir='data/mushroom_data',
    output_dir='data/processed',
    test_size=0.2,
    img_size=(224, 224)
)

# Verify processed data structure
!tree data/processed -L 3

## 4. Model Training
Train the CNN model on the processed dataset

In [ ]:
from src.train import train_model

# Train the model
train_model()

# Display training history
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = mpimg.imread('training_history.png')
plt.figure(figsize=(15, 5))
plt.imshow(img)
plt.axis('off')
plt.show()

# Show training metrics
print("\nTraining Metrics:")
!cat training_metrics.txt

## 5. Model Evaluation and Prediction
Test the trained model on sample images

In [ ]:
from src.predict import predict_mushroom
import random
from pathlib import Path

def test_random_images(num_samples=3):
    test_dir = Path('data/processed/test')
    
    for category in ['edible', 'poisonous']:
        print(f"\nTesting {category} mushrooms:")
        category_path = test_dir / category
        image_files = list(category_path.glob('*.[Jj][Pp][Gg]'))
        
        for _ in range(num_samples):
            test_image = random.choice(image_files)
            print(f"\nImage: {test_image.name}")
            prediction, confidence = predict_mushroom(
                'mushroom_classifier.keras',
                str(test_image)
            )
            print(f"Predicted: {prediction}")
            print(f"Confidence: {confidence:.2%}")

# Test model on random images
test_random_images()

## 6. Save Model and Results

In [ ]:
# Save results to Google Drive if in Colab
if IN_COLAB:
    !cp mushroom_classifier.keras "{RESULTS_DIR}/"
    !cp best_model.keras "{RESULTS_DIR}/"
    !cp training_history.png "{RESULTS_DIR}/"
    !cp training_metrics.txt "{RESULTS_DIR}/"
    !cp training_log.csv "{RESULTS_DIR}/"
    print(f"Model and results saved to Google Drive: {RESULTS_DIR}")
else:
    !mkdir -p results
    !cp mushroom_classifier.keras results/
    !cp best_model.keras results/
    !cp training_history.png results/
    !cp training_metrics.txt results/
    !cp training_log.csv results/
    print("Model and results saved in 'results' directory")